#### Secondary Postprocessing Suite

Figures from the non-trivial secondary data $\{\langle c\rangle_x, \langle s\rangle_x, h, \}$ 

In [ ]:
%load_ext autoreload

%autoreload 2

import numpy as np
from functools import partial 
from lucifex.io import find_dataset, find_by_parameters, ld, postprocess
from lucifex.fdm import NumericSeries, GridSeries
from lucifex.viz import detexify
from lucifex.utils import as_indices

DATASET = find_dataset(repo='CarbonDissolution', data='_data/convection_reaction', include='Ra=*')

from co2_dissolution_pkg.postprocess.plot import plot_colormaps, plot_timeseries, plot_horizontal_average
from co2_dissolution_pkg.postprocess.compute import compute_saturation_front, compute_events_umax, compute_subdomain_averages

PARAMETERS = 'PARAMETERS.txt'
FUNCTION_SERIES = 'FunctionSeries.h5'
CONSTANT_SERIES = 'ConstantSeries.h5'
GRID_SERIES = 'GridSeries.npz'
NUMERIC_SERIES = 'NumericSeries.npz'
POSTPROCESS = 'PostProcess'
FIGURES = 'figures'

DATASET = find_dataset(repo='CarbonDissolution', data='_data/convection_reaction', include='Ra=*')

Ra = 800
Da = 1000
sr = 0.1 
parameters = dict(Ra=Ra, Da=Da, sr=sr)
directory = find_by_parameters(DATASET.directories, PARAMETERS, parameters)

In [2]:
ALL = False
if ALL:
    dir_arg = DATASET.directories
else:
    dir_arg = directory

#### Exploratory Postprocessing

In [ ]:
from co2_dissolution_pkg.postprocess.compute import *
from co2_dissolution_pkg.postprocess.plot import *


In [4]:
# finger trajectories

window = (2.0, 9.0)
alpha = 0.2
adaptive = False
fig_name = f'cPlumes({window[0]}<t<{window[1]};alpha={alpha},adaptive={adaptive})'
plot_finger_trajectories(dir_arg, fig_name, FIGURES)(
    ld(('c', GRID_SERIES)),
    window,
    alpha,
    adaptive,
    condition=100,
)

Postprocess result saved.


In [ ]:
# plume velocity

window = (2.0, 9.0)
alpha = 0.2
adaptive = False
condition = 100
def make_avg_finger_velocity(
    c: GridSeries,
) -> NumericSeries:
    return compute_plume_velocity_average(c, window, alpha, adaptive, condition=condition)

fig_name = f'uPlume({window[0]}<t<{window[1]};alpha={alpha},adaptive={adaptive})'
plot_timeseries(dir_arg, fig_name, FIGURES)(
    ld(make_avg_finger_velocity, ('c', GRID_SERIES)),
    label='$u_P^{\downarrow}$',
)

Postprocess result saved.


: 

#### Established Postprocessing

In [ ]:
# utilities
make_suptitle = lambda Ra, Da, sr: f'$(Ra, Da, s_r)=({Ra}, {Da}, {sr})$'

In [ ]:
# maximum velocity events
compute_events_umax(dir_arg, POSTPROCESS, name=('umax_tonset', 'umax_tmax', 'umax_tshut'))(
    ld(('umax', NUMERIC_SERIES)),
)

In [ ]:
# horizontally-averaged concentration
fig_name = '<c>x(t)'
plot_horizontal_average(dir_arg, fig_name, FIGURES)(
    ld(('c', GRID_SERIES)),
    20,
    x_lims=(0,1),
    y_lims=(0,1),
    suptitle=ld(make_suptitle, (float, 'Ra', PARAMETERS), (float, 'Da', PARAMETERS), (float, 'sr', PARAMETERS)),
)

In [ ]:
# horizontally-averaged saturation
make_xlims = lambda Sr: (0, Sr)
make_ylims = lambda h0, Ly: (h0 - 0.5*(Ly - h0), Ly)
fig_name = '<s>x(t)'
plot_horizontal_average(dir_arg, fig_name, FIGURES)(
    ld(('s', GRID_SERIES)),
    20,
    x_lims=ld(make_xlims, (float, 'sr', PARAMETERS)), 
    y_lims=ld(make_ylims, (float, 'h0', PARAMETERS), (float, 'Ly', PARAMETERS)),
    suptitle=ld(make_suptitle, (float, 'Ra', PARAMETERS), (float, 'Da', PARAMETERS), (float, 'sr', PARAMETERS)), 
)

In [ ]:
# # front height
# def make_h(s: GridSeries, eps=0.01, h0=None):
#     hxt = compute_saturation_front(s, eps)
#     if h0:
#         h0_correction = h0 - np.mean(hxt.series[0][1])
#     else:
#         h0_correction = 0.0
#     return NumericSeries([h0_correction + np.mean(i[1]) for i in hxt.series], hxt.time_series, 'hbraket')


make_h = partial(compute_saturation_front, s_value=0.01)
plot_timeseries(dir_arg, 'h(t)', FIGURES)(
    ld(make_h, ('s', GRID_SERIES)),
    label='$\langle h\\rangle_x$'
)

In [ ]:
# subdomain-averaged concentrations

def make_averages(c, s, eps=0.01, h0=None):
    h = make_h(s, eps, h0)
    return compute_subdomain_averages(c, h)

make_plus = lambda c, s, h0=None: make_averages(c, s, h0)[0]
make_minus = lambda c, s, h0=None: make_averages(c, s, h0)[1]

plot_timeseries(dir_arg, 'cplus(t)', FIGURES)(
    ld(make_plus, ('c', GRID_SERIES), ('s', GRID_SERIES)),
    label='$c^+$'
)
plot_timeseries(dir_arg, 'cminus(t)', FIGURES)(
    ld(make_minus, ('c', GRID_SERIES), ('s', GRID_SERIES)),
    label='$c^-$'
)

In [ ]:
# subdomain-averaged saturation

plot_timeseries(dir_arg, 'splus(t)', FIGURES)(
    ld(make_plus, ('s', GRID_SERIES), ('s', GRID_SERIES)),
    label='$s^+$'
)

In [ ]:
# saturation with front contours
make_ylims = lambda h0, Ly: (h0, Ly)
make_colorbar = lambda Sr: (0, Sr)
def make_contours(s: GridSeries, s_value: float):
    hxt = compute_saturation_front(s, s_value)
    return [(i[0], i[1]) for i in hxt.series]

eps = 0.01
fig_title = lambda fig: f'{detexify(fig.axes[0].get_title())}_eps={eps}_front'
plot_colormaps(dir_arg, fig_title, FIGURES)(
    ld(('s', GRID_SERIES)), 
    10, 
    ld(partial(make_contours, eps=eps), (GridSeries, 's', GRID_SERIES)),
    colorbar=ld(make_colorbar, (float, 'Sr', PARAMETERS)),
    y_lims=ld(make_ylims, (float, 'h0', PARAMETERS), (float, 'Ly', PARAMETERS)),
    aspect='auto',
    color='cyan',
)

In [ ]:
# intricate custom plot

@postprocess
def plot_plumes(c):
    t = 10
    fig_axs = plot_colormaps(c, t)
    for fig, ax in fig_axs:
        ...
    return fig_axs


plot_plumes(dir_arg)